# **Keras code for CIFAR10 classification by CNN**

This simple CNN generates about 36% test accuracy for 100 training epochs.

### **Import Keras necessary package**

In [0]:
# import os

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adagrad, RMSprop, SGD
# from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### **Define all settings**

In [0]:
num_classes = 10
data_augmentation = True

In [0]:
# save_dir = os.path.join(os.getcwd(), 'saved_models')
# model_name = 'keras_cifar10_trained_model.h5'

# if not os.path.isdir(save_dir):
#    os.makedirs(save_dir)

# model_path = os.path.join(save_dir, model_name)

### **The data, split between train and test sets:**

In [8]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train.shape, x_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

### **Convert class vectors to binary class matrices.**

In [9]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

y_train.shape, y_test.shape

((50000, 10), (10000, 10))

### **Define CNN architecture**

In [0]:
lr_decay = 1e-4

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(lr_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(lr_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(lr_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(lr_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(lr_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(lr_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

### **Model compilation**

In [11]:
opt_lr = 0.001
opt_decay = 1e-6

optimizer = RMSprop(lr=opt_lr, decay=opt_decay)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0

### **Data augmentation**

In [12]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        #rotation_range=20,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
        #horizontal_flip=True
        )
    datagen.fit(x_train)

x_train.shape, x_test.shape

Using real-time data augmentation.


((50000, 32, 32, 3), (10000, 32, 32, 3))

### **Train the model**

In [13]:
batch_size = 64
epochs = 24

model.fit(x_train, y_train,
                batch_size=batch_size,
            steps_per_epoch=x_train.shape[0]//batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),)

Epoch 1/24
782/782 [==============================] - 21s 27ms/step - loss: 1.7291 - accuracy: 0.4842 - val_loss: 1.1182 - val_accuracy: 0.6330
Epoch 2/24
782/782 [==============================] - 21s 27ms/step - loss: 1.1060 - accuracy: 0.6517 - val_loss: 1.0224 - val_accuracy: 0.6841
Epoch 3/24
782/782 [==============================] - 21s 26ms/step - loss: 0.9162 - accuracy: 0.7137 - val_loss: 0.9079 - val_accuracy: 0.7181
Epoch 4/24
782/782 [==============================] - 21s 27ms/step - loss: 0.8213 - accuracy: 0.7487 - val_loss: 1.7164 - val_accuracy: 0.5553
Epoch 5/24
782/782 [==============================] - 21s 27ms/step - loss: 0.7516 - accuracy: 0.7725 - val_loss: 0.8261 - val_accuracy: 0.7497
Epoch 6/24
782/782 [==============================] - 21s 27ms/step - loss: 0.6996 - accuracy: 0.7958 - val_loss: 0.8006 - val_accuracy: 0.7738
Epoch 7/24
782/782 [==============================] - 21s 27ms/step - loss: 0.6699 - accuracy: 0.8090 - val_loss: 0.7511 - val_accuracy:

### **Save model and weights**

In [0]:
# model.save(model_path)

# model_path

In [0]:
# model = load_model(model_path)

# model_path

### **Score trained model.**

In [16]:
scores = model.evaluate(x_test, y_test, batch_size=128)

scores

79/79 [==============================] - 1s 12ms/step - loss: 0.6538 - accuracy: 0.8418


[0.653809130191803, 0.8417999744415283]